# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
directory <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/output"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248-summary.csv
<chr>,<chr>,<chr>,<chr>
chunk1,1001000,1004000,1005000
chunk2,1001999,1004999,1005999
tag,libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244,libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247,libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248
time_started,2024-02-01 16:42:47,2024-02-01 16:42:50,2024-02-01 16:42:54
time_finished,2024-02-01 18:10:46,2024-02-01 18:54:29,2024-02-01 18:46:42
runtime,01:27:59,02:11:38,02:03:48
type_CPU,AMD EPYC 7713 64-Core Processor,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz
amount_RAM,0.983715858310461,0.491538614034653,0.491538614034653
physical_cores,128,64,64


## Compute core-hours per test

In [3]:
windows <- c(1000,2000,5000,10000,20000,50000,100000,500000)
n_windows <- length(windows)
alphas <- seq(0, 1, .25) # left out 0 on first run
n_alphas <- length(alphas)
chunks <- 1000
n_tests <- n_windows * n_alphas * chunks
n_tests

[1] 40000

In [4]:
alphas

[1] 0.00 0.25 0.50 0.75 1.00

In [5]:
colnames(summary_table)

[1] "Parameter"                                                                                                   
[2] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244-summary.csv"
[3] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247-summary.csv"
[4] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248-summary.csv"

In [6]:
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248-summary.csv
<chr>,<chr>,<chr>,<chr>
chunk1,1001000,1004000,1005000
chunk2,1001999,1004999,1005999
tag,libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244,libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247,libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248
time_started,2024-02-01 16:42:47,2024-02-01 16:42:50,2024-02-01 16:42:54
time_finished,2024-02-01 18:10:46,2024-02-01 18:54:29,2024-02-01 18:46:42
runtime,01:27:59,02:11:38,02:03:48
type_CPU,AMD EPYC 7713 64-Core Processor,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz
amount_RAM,0.983715858310461,0.491538614034653,0.491538614034653
physical_cores,128,64,64


In [7]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]
summary_table$number_cores <- 1

In [8]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [9]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [10]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / n_tests
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / n_tests

In [11]:
summary_table

,chunk1,chunk2,tag,time_started,time_finished,runtime,type_CPU,amount_RAM,physical_cores,scaffold_ID,number_cores,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244-summary.csv,1001000,1001999,libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244,2024-02-01 16:42:47,2024-02-01 18:10:46,01:27:59,AMD EPYC 7713 64-Core Processor,0.983715858310461,128,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1001000-1001999-static-1corestotal-1corepera-20240201-164244,1,5279,1.466389,1.466389,5279,3.665972e-05,0.131975
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247-summary.csv,1004000,1004999,libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247,2024-02-01 16:42:50,2024-02-01 18:54:29,02:11:38,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz,0.491538614034653,64,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1004000-1004999-static-1corestotal-1corepera-20240201-164247,1,7898,2.193889,2.193889,7898,5.484722e-05,0.197450
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248-summary.csv,1005000,1005999,libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248,2024-02-01 16:42:54,2024-02-01 18:46:42,02:03:48,Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz,0.491538614034653,64,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1005000-1005999-static-1corestotal-1corepera-20240201-164248,1,7428,2.063333,2.063333,7428,5.158333e-05,0.185700


In [ ]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [ ]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a2.csv")

## How long for whole dataset?

In [13]:
n_sites <- 26000000

In [14]:
n_sites*0.0001107989 # From 1000000-1001000

[1] 2880.771

In [15]:
n_sites*mean(summary_table$core_hours_per_test)

[1] 1240.116